In [24]:
!pip install -q crewai langchain langchain_core langchain_community langchain-openai wikipedia tavily-python google-search-results

  Preparing metadata (setup.py) ... done


In [4]:
from pydantic import BaseModel
from typing import List, Dict, Any

class FlightRequest(BaseModel):
    origin: str
    destination: str
    date: str  # Format: 'YYYY-MM-DD'
    available_budget: float

class FlightOption(BaseModel):
    airline: str
    flight_number: str
    departure_time: str  # Format: 'YYYY-MM-DD HH:MM'
    arrival_time: str    # Format: 'YYYY-MM-DD HH:MM'
    duration: str        # Format: 'HH:MM'
    price: float
    stops: int
    layover_cities: List[str]
    aircraft_type: str
    seat_class: str
    amenities: List[str]
    booking_link: str

class FlightAnalysis(BaseModel):
    recommended_flights: List[FlightOption]
    total_cost_savings: float
    cost_breakdown: Dict[str, float]  # Mapping flight numbers to cost
    alternative_routes: List[FlightOption]
    analysis_details: Dict[str, Any]  # Details about analysis criteria and methods
    recommendations: List[str]
    potential_challenges: List[str]
    additional_notes: str
    resources: List[str]
    contact_information: str
    legal_disclaimer: str

In [19]:
from google.colab import userdata
import os
os.environ["OPENAI_API_KEY"] = userdata.get('OPENAI_API_KEY')
os.environ["SERPAPI_API_KEY"] = userdata.get('SERPAPI_API_KEY')
os.environ["TAVILY_API_KEY"] = userdata.get('TAVILY_API_KEY')

In [21]:
from crewai import Agent
from textwrap import dedent
from langchain_openai import ChatOpenAI
from langchain.tools import Tool
from langchain_community.tools import TavilySearchResults
from langchain_community.utilities import WikipediaAPIWrapper
from langchain_community.tools import WikipediaQueryRun
from langchain_community.utilities import SerpAPIWrapper

class CustomAgents:
    def __init__(self):
        self.OpenAIGPT4Mini = ChatOpenAI(model="gpt-4o-mini", temperature=0)
        self.OpenAIGPT4 = ChatOpenAI(model="gpt-4o", temperature=0)
        self.tools = [Tool(
                          name="Wikipedia",
                          func=WikipediaQueryRun(api_wrapper=WikipediaAPIWrapper()).run,
                          description="Useful for when you need to answer questions about current events. You should ask targeted questions."
                      ),
                      TavilySearchResults(
                        max_results=5,
                        include_answer=True,
                        include_raw_content=True
                      ),
                      Tool(
                          name="SerpAPI",
                          func=SerpAPIWrapper().run,
                          description="Useful for when you need to search for external info. in the Internet."
                      )]

    def flight_data_collector_agent(self):
        return Agent(
            role="Flight Data Collector",
            backstory=dedent("""You are an expert in gathering real-time flight information based on user input. You utilize tools to collect accurate and up-to-date flight data."""),
            goal=dedent("""Collect available flights from the specified origin to the destination on the given date within the user's available budget, including detailed flight information as per the FlightOption schema."""),
            tools=self.tools,
            allow_delegation=False,
            verbose=True,
            llm=self.OpenAIGPT4Mini,
        )

    def flight_optimizer_agent(self):
        return Agent(
            role="Flight Optimizer",
            backstory=dedent("""You specialize in analyzing flight options to recommend the best flights considering factors like price, duration, stops, and amenities."""),
            goal=dedent("""Analyze the collected flight data and recommend the best flight options to the user, including detailed justifications and potential cost savings."""),
            tools=self.tools,
            allow_delegation=False,
            verbose=True,
            llm=self.OpenAIGPT4,
        )

    def cost_analysis_agent(self):
        return Agent(
            role="Cost Analysis Agent",
            backstory=dedent("""You provide insights on how the user can save costs on their flights and offer detailed cost breakdowns and alternative options."""),
            goal=dedent("""Identify potential cost savings, provide a cost breakdown, and suggest alternative routes or options that could reduce expenses while meeting the user's needs."""),
            tools=self.tools,
            allow_delegation=False,
            verbose=True,
            llm=self.OpenAIGPT4Mini,
        )

    def report_generator_agent(self):
        return Agent(
            role="Report Generator",
            backstory=dedent("""You compile all the information from the previous analyses and provide a comprehensive flight analysis report to the user, adhering to the detailed FlightAnalysis schema."""),
            goal=dedent("""Using the collected flight data and analyses, provide a comprehensive flight analysis report matching the FlightAnalysis schema, ensuring all fields are populated with detailed and accurate information."""),
            tools=[],
            allow_delegation=False,
            verbose=True,
            llm=self.OpenAIGPT4,
        )


In [22]:
from crewai import Task
from textwrap import dedent

class CustomTasks:
    def __init__(self):
        pass

    def flight_data_collection_task(self, agent, flight_request: FlightRequest):
        return Task(
            description=dedent(f"""
                Collect all available flights from **{flight_request.origin}** to **{flight_request.destination}** on **{flight_request.date}** within the budget of **${flight_request.available_budget}**.
                Provide detailed flight options including all attributes defined in the FlightOption schema.
                **Use the tools provided to gather accurate and up-to-date flight information.**
            """),
            agent=agent,
            expected_output="A list of available flights that meet the user's criteria, with detailed information per the FlightOption schema.",
        )

    def flight_optimization_task(self, agent, flight_request: FlightRequest):
        return Task(
            description=dedent(f"""
                Analyze the collected flight options and recommend the best flights considering factors such as price, duration, number of stops, layover cities, aircraft type, seat class, and amenities.
                Provide detailed explanations for your recommendations, including potential cost savings and benefits.
                **Use the tools provided to enhance your analysis.**
            """),
            agent=agent,
            expected_output="A set of recommended flights with detailed explanations and justifications.",
        )

    def cost_analysis_task(self, agent, flight_request: FlightRequest):
        return Task(
            description=dedent(f"""
                Provide a detailed cost analysis for the user's flight from **{flight_request.origin}** to **{flight_request.destination}** on **{flight_request.date}**.
                Include a cost breakdown per flight option, total potential cost savings, and suggest alternative routes or options that could further reduce costs.
                **Use the tools provided to support your recommendations.**
            """),
            agent=agent,
            expected_output="Detailed cost analysis with cost breakdown, total savings, and alternative suggestions.",
        )

    def flight_analysis_report_task(self, agent, flight_request: FlightRequest):
        flight_analysis_schema = FlightAnalysis.schema_json(indent=2)
        return Task(
            description=dedent(f"""
                Compile a comprehensive flight analysis report based on the collected data and analyses.
                Include the following in your report:
                - **Recommended Flights**: A detailed list of recommended flights, including all attributes in the FlightOption schema.
                - **Total Cost Savings**: The total amount of potential savings identified.
                - **Cost Breakdown**: A detailed breakdown of costs per flight option.
                - **Alternative Routes**: Suggested alternative routes or flights that could offer additional benefits.
                - **Analysis Details**: Information on the criteria and methods used in the analysis.
                - **Recommendations**: Actionable recommendations for the user.
                - **Potential Challenges**: Any challenges or considerations the user should be aware of.
                - **Additional Notes**, **Resources**, **Contact Information**, **Legal Disclaimer**: Provide any additional relevant information.

                Provide your output in **JSON format** matching the **FlightAnalysis** schema below.

                **FlightAnalysis Schema**:

                {flight_analysis_schema}
            """),
            agent=agent,
            expected_output=f"The flight analysis report in JSON format matching the FlightAnalysis schema: {flight_analysis_schema}",
        )

In [25]:
from crewai import Crew
from textwrap import dedent

class FlyOpsAnalystCrew:
    def __init__(self, origin, destination, date, available_budget):
        self.flight_request = FlightRequest(
            origin=origin,
            destination=destination,
            date=date,
            available_budget=available_budget
        )
        self.agents = CustomAgents()
        self.tasks = CustomTasks()

    def run(self):
        # Define agents
        flight_data_collector_agent = self.agents.flight_data_collector_agent()
        flight_optimizer_agent = self.agents.flight_optimizer_agent()
        cost_analysis_agent = self.agents.cost_analysis_agent()
        report_generator_agent = self.agents.report_generator_agent()

        # Define tasks
        flight_data_collection_task = self.tasks.flight_data_collection_task(flight_data_collector_agent, self.flight_request)
        flight_optimization_task = self.tasks.flight_optimization_task(flight_optimizer_agent, self.flight_request)
        cost_analysis_task = self.tasks.cost_analysis_task(cost_analysis_agent, self.flight_request)
        flight_analysis_report_task = self.tasks.flight_analysis_report_task(report_generator_agent, self.flight_request)

        # Create the crew
        crew = Crew(
            agents=[
                flight_data_collector_agent,
                flight_optimizer_agent,
                cost_analysis_agent,
                report_generator_agent,
            ],
            tasks=[
                flight_data_collection_task,
                flight_optimization_task,
                cost_analysis_task,
                flight_analysis_report_task,
            ],
            verbose=True,
        )

        result = crew.kickoff()
        return result

if __name__ == "__main__":
    print("## Welcome to the FlyOps Analyst")
    print("-------------------------------------------------------")
    origin = input("Enter the origin airport: ")
    destination = input("Enter the destination airport: ")
    date = input("Enter the date of travel (YYYY-MM-DD): ")
    available_budget = float(input("Enter your available budget in USD: "))

    analyst = FlyOpsAnalystCrew(
        origin=origin.strip(),
        destination=destination.strip(),
        date=date.strip(),
        available_budget=available_budget
    )
    results = analyst.run()
    print("\n\n########################")
    print("## Here is your flight analysis result:")
    print("########################\n")
    print(results)

## Welcome to the FlyOps Analyst
-------------------------------------------------------
Enter the origin airport: New York
Enter the destination airport: Lima
Enter the date of travel (YYYY-MM-DD): 2024-10-10
Enter your available budget in USD: 1000
# Agent: Flight Data Collector
## Task: 
Collect all available flights from **New York** to **Lima** on **2024-10-10** within the budget of **$1000.0**.
Provide detailed flight options including all attributes defined in the FlightOption schema.
**Use the tools provided to gather accurate and up-to-date flight information.**



# Agent: Flight Data Collector
## Thought: I need to gather information about available flights from New York to Lima on the specified date within the budget. I will start by searching for flights that match these criteria.
## Using tool: tavily_search_results_json
## Tool Input: 
"{\"query\": \"flights from New York to Lima on 2024-10-10 under $1000\"}"
## Tool Output: 
[{'url': 'https://www.kayak.com/flight-routes

In [26]:
results

CrewOutput(raw='{\n  "recommended_flights": [\n    {\n      "airline": "Delta Airlines",\n      "flight_number": "DL123",\n      "departure_time": "2024-10-10T08:00:00",\n      "arrival_time": "2024-10-10T16:00:00",\n      "duration": "8 hours",\n      "price": 447,\n      "stops": 0,\n      "layover_cities": [],\n      "aircraft_type": "Boeing 767",\n      "seat_class": "Economy",\n      "amenities": ["In-flight entertainment", "Wi-Fi", "Complimentary meals"],\n      "booking_link": "https://www.delta.com/book"\n    },\n    {\n      "airline": "LATAM Airlines",\n      "flight_number": "LA456",\n      "departure_time": "2024-10-10T09:00:00",\n      "arrival_time": "2024-10-10T17:00:00",\n      "duration": "8 hours",\n      "price": 346,\n      "stops": 0,\n      "layover_cities": [],\n      "aircraft_type": "Airbus A350",\n      "seat_class": "Economy",\n      "amenities": ["In-flight entertainment", "Wi-Fi"],\n      "booking_link": "https://www.latam.com/book"\n    },\n    {\n      "a